In [1]:
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from random import randrange, choices, sample
import random

In [2]:
# fix (or add) agent based datacollector
# fix step id for model-based datacollector

# Functions

In [3]:
def generate_probability():
    return randrange(100)

def generate_weights():
    pass

def generate_score():
    return randrange(100)

def generate_distribution(n, total):
    dividers = sorted(sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]



class Behavior:
    ActionSuccessProbability = generate_probability()
    TakeActionProbability = generate_probability()
    ActionSelectionWeights = generate_weights()
    MakeMoveProbability = generate_probability()
    BreedSuccessProbability = generate_probability()
    BreedActionProbability = generate_probability()
    MoveDirectionDistribution = generate_distribution(9, 100)
    AwarenessScore = generate_score()
    ActionOrderDistribution = generate_distribution(3, 100)
    ActionPrivilegeScore = generate_score()
    HappinessScore = generate_score()
    EnvironmentScore = generate_score()
    
    def fate(self, weight):
        fate_list = [True, False]
        weight_var = (weight, 100-weight)

        return random.choices(fate_list, weights=weight_var, k=len(fate_list))[0]

In [4]:
class RandomWalker(mesa.Agent):
    """
    Class implementing random walker methods in a generalized manner.
    Not intended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        moore: If True, may move in all 8 directions.
                Otherwise, only up, down, left, right.
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(
            self.pos, self.moore, True)

        # move check
        if len(next_moves) > 0:

            if self.behavior.fate(self.behavior.MakeMoveProbability):
                self.move = self.random.choices(
                    next_moves, weights=self.behavior.MoveDirectionDistribution, k=1)[0]
                self.model.grid.move_agent(self, self.move)

            else:
                pass

        else:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

In [5]:
class Dooder(RandomWalker):
    """
    """
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.behavior = Behavior()
        self.move = None
    
    def kill(self, agent):
        self.model.grid.remove_agent(agent)
        self.model.schedule.remove(agent)

    def die(self):
        self.model.grid.remove_agent(self)
        self.model.schedule.remove(self)

    def step(self):
        # move --> action
        print(self.unique_id)
        self.random_move()

In [6]:
class Simulation(Model):
    """
    Wolf-Sheep Predation Model
    """

    height = 20
    width = 20

    initial_agents = 10
    verbose = True  # Print-monitoring

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(self,
                 width=20,
                 height=20,
                 initial_agents=initial_agents,
                 verbose=verbose):
        """
        """
        super().__init__()
        # Set parameters
        self.width = width
        self.height = height
        self.initial_agents = initial_agents
        self.verbose = verbose

        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(self.width, self.height, torus=True)
        self.datacollector = DataCollector(model_reporters={
            "Dooders":
            lambda m: m.schedule.get_agent_count()
        },
        agent_reporters={'Move': 'move'})
        self.agent_count = 0

        # Create agents:
        for i in range(self.initial_agents):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            happiness = self.random.randrange(10, 90)
            dooder = Dooder(self.next_id(), (x, y), self)
            self.grid.place_agent(dooder, (x, y))
            self.schedule.add(dooder)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print([self.schedule.time, self.schedule.get_agent_count()])

    def run_model(self, step_count=10):

        if self.verbose:
            print("Initial number dooders: ", self.schedule.get_agent_count())

        for i in range(step_count):
            self.agent_count = self.schedule.get_agent_count()
            # running status
            # metrics
            self.step()
            break

        if self.verbose:
            print("")
            print("Final number dooders: ", self.schedule.get_agent_count())

In [7]:
test = Simulation()

In [27]:
test.run_model()

Initial number dooders:  10
9
7
6
10
4
8
5
1
2
3
[4, 10]

Final number dooders:  10


In [28]:
test.datacollector.get_model_vars_dataframe()

,Dooders
0,10
1,10
2,10
3,10
4,10


In [29]:
test.datacollector.get_agent_vars_dataframe()

Move
Step AgentID          
0    1            None
     2            None
     3            None
     4            None
     5            None
     6            None
     7            None
     8            None
     9            None
     10           None
1    1        (15, 18)
     2         (4, 16)
     3         (9, 12)
     4        (11, 18)
     5         (1, 11)
     6         (15, 2)
     7         (13, 4)
     8          (3, 6)
     9         (1, 14)
     10       (19, 15)
2    1        (16, 19)
     2         (4, 17)
     3         (9, 12)
     4        (11, 18)
     5         (2, 10)
     6         (14, 2)
     7         (12, 4)
     8          (2, 6)
     9         (0, 14)
     10        (0, 15)
3    1         (16, 0)
     2         (4, 16)
     3         (8, 12)
     4        (10, 18)
     5         (1, 10)
     6         (15, 1)
     7         (13, 3)
     8          (2, 7)
     9        (19, 13)
     10        (1, 16)
4    1         (15, 1)
     2         (5, 17)
     3         (8, 13)
     4        (11, 19)
     5         (2, 11)
     6         (15, 1)
     7         (13, 3)
     8          (3, 8)
     9         (0, 13)
     10        (1, 17)

In [11]:
test.current_id

10

In [12]:
test.run_model()

Initial number dooders:  10
10
4
3
9
6
5
1
7
2
8
[2, 10]

Final number dooders:  10


In [13]:
test.running

True

In [14]:
# How to stop a model in progress
# Does the class save actions taken?

In [15]:
test.schedule.agents

In [16]:
test.schedule.agents[1].pos

(4, 17)

In [17]:
test.grid.get_neighbors((14,2), moore=True)

[]

In [18]:
test.datacollector.get_agent_vars_dataframe()

Move
Step AgentID          
0    1            None
     2            None
     3            None
     4            None
     5            None
     6            None
     7            None
     8            None
     9            None
     10           None
1    1        (15, 18)
     2         (4, 16)
     3         (9, 12)
     4        (11, 18)
     5         (1, 11)
     6         (15, 2)
     7         (13, 4)
     8          (3, 6)
     9         (1, 14)
     10       (19, 15)
2    1        (16, 19)
     2         (4, 17)
     3         (9, 12)
     4        (11, 18)
     5         (2, 10)
     6         (14, 2)
     7         (12, 4)
     8          (2, 6)
     9         (0, 14)
     10        (0, 15)

In [19]:
test.datacollector.get_model_vars_dataframe()

,Dooders
0,10
1,10
2,10


In [20]:
test.datacollector.tables

{}

In [21]:
test.grid.torus

True

# Testing viz input

In [22]:
data_colector = getattr(test, 'datacollector')

In [23]:
data_colector.get_agent_vars_dataframe().astype('float')

ValueError: setting an array element with a sequence.

In [ ]:
test.schedule.step()

In [ ]:
from collections import defaultdict

In [ ]:
grid_state = defaultdict(list)
for x in range(test.grid.width):
    for y in range(test.grid.height):
        cell_objects = test.grid.get_cell_list_contents([(x, y)])
        for obj in cell_objects:
            portrayal = self.portrayal_method(obj)
            if portrayal:
                portrayal["x"] = x
                portrayal["y"] = y
                grid_state[portrayal["Layer"]].append(portrayal)

In [ ]:
grid_state = defaultdict(list)
for x in range(test.grid.width):
    for y in range(test.grid.height):
        cell_objects = test.grid.get_cell_list_contents([(x, y)])
        if cell_objects:
            for obj in cell_objects:
                print(obj.pos)


In [ ]:
ddd.pos

In [ ]:
data_collector_name = 'datacollector'
series = [{"Label": "Dooders", "Color": "Black"},]

def render(model):
    current_values = []
    data_collector = getattr(model, data_collector_name)

    for s in series:
        name = s["Label"]
        try:
            val = data_collector.model_vars[name][-1]  # Latest value
        except (IndexError, KeyError):
            val = 0
        current_values.append(val)
    return current_values

In [ ]:
render(test)

In [ ]:
test.schedule.agents[1].kill(test.schedule.agents[1])

In [32]:
import math

In [33]:
def direction_lookup(destination_x, origin_x, destination_y, origin_y):

    deltaX = destination_x - origin_x

    deltaY = destination_y - origin_y

    degrees_temp = math.atan2(deltaX, deltaY)/math.pi*180

    if degrees_temp < 0:

        degrees_final = 360 + degrees_temp

    else:

        degrees_final = degrees_temp

    compass_brackets = ["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"]

    compass_lookup = round(degrees_final / 45)

    return compass_brackets[compass_lookup], degrees_final

In [35]:
direction_lookup(7,2,2,5)

('SE', 120.96375653207352)

In [40]:
def get_direction(origin, destination):
    compass_brackets = ["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"]
    deltaX = destination[0] - origin[0]
    deltaY = destination[1] - origin[1]
    
    degrees = math.atan2(deltaX, deltaY)/math.pi*180
    if degrees < 0:
        degrees_final = 360 + degrees_temp
    else:
        degrees_final = degrees
        
    compass_lookup = round(degrees_final / 45)

    return compass_brackets[compass_lookup]

In [45]:
origin = (7,3)
destination = (7,3)

In [46]:
get_direction(origin, destination)

'N'